In [32]:
import dataclasses

from joshpyutil import mpl
import numpy as np
import scipy.interpolate
import progressbar


t_max = 4
t_rain_stop = 3
lax_friedrichs_dt = 0.005
n = 200
terrain_r = 0.2
terrain_h = 0.3
gamma = 0.1
R = 0.
g = 4

x, dx = np.linspace(0, 1, n, retstep=True)
x_mid = (x[1:] + x[:-1]) / 2


def grad(field):
    spline = scipy.interpolate.Akima1DInterpolator(x, field)
    return spline.derivative()(x)


terrain = (
    -1.5 * terrain_h * np.exp(-(x - 1/2)**2 / terrain_r**2)
    + 0.3 * terrain_h * np.sin(14 * np.pi * (x - 1/2) + 1)
)
grad_terrain = grad(terrain)
water = np.zeros(n)
moment = np.zeros(n)


@dataclasses.dataclass
class Fields:
    height: np.ndarray[float]
    momentum: np.ndarray[float]

    def pack(self) -> np.ndarray[float]:
        stacked = np.stack([self.height, self.momentum], axis=1)
        return np.ravel(stacked)

    @classmethod
    def unpack(cls, packed: np.ndarray[float]) -> 'Fields':
        stacked = packed.reshape((n, 2))
        return cls(height=stacked[:, 0], momentum=stacked[:, 1])

    @classmethod
    def zeros(cls) -> 'Fields':
        return cls(height=np.zeros(n), momentum=np.zeros(n))


def lax_friedrichs_diff(field: np.ndarray[float]) -> np.ndarray[float]:
    return 1 / 2 / lax_friedrichs_dt * (field[2:] + field[:-2] - 2 * field[1:-1])


#   0   1    n-2 n-1
# |-*-|-*-|...|-*-|
#     0   1  n-2


def interpolate_mid(field: np.ndarray[float]) -> np.ndarray[float]:
    interpolant = scipy.interpolate.InterpolatedUnivariateSpline(x, field, k=1)
    return interpolant(x_mid)


def time_deriv_func(t, packed) -> np.ndarray:
    fields = Fields.unpack(packed)
    
    grad_height = grad(fields.height)
    accel_grav = -g / (1 + (grad_height + grad_terrain)**2) * (grad_height + grad_terrain)
    
    height_flux = interpolate_mid(fields.momentum.copy())
    
    velocity = fields.momentum.copy()
    tiny_mask = fields.height < 0.001
    velocity[tiny_mask] = 0
    velocity[~tiny_mask] /= fields.height[~tiny_mask]
    momentum_flux = interpolate_mid(fields.momentum * velocity)
    
    fields_time_deriv = Fields.zeros()
    
    if t < t_rain_stop:
        rain_rate = R
    else:
        rain_rate = 0
    
    fields_time_deriv.height[1:-1] = (
        -1/dx * (height_flux[1:] - height_flux[:-1])
        + lax_friedrichs_diff(fields.height)
        + rain_rate * (
            np.exp(-((x[1:-1] - 1/2) / terrain_r)**60)
            # * np.sin((np.pi * x[1:-1] + np.pi * t) / terrain_r)**2
        )
    )
    # fields_time_deriv.height[0] = fields_time_deriv.height[1]
    # fields_time_deriv.height[-1] = fields_time_deriv.height[-2]
    # fields_time_deriv.height[0] = water_wave_speed * grad_height[0]
    # fields_time_deriv.height[-1] = -water_wave_speed * grad_height[-1]
    
    negative_mask = fields.height < -0.0001
    fields_time_deriv.height[negative_mask] = -fields.height[negative_mask]
    
    fields_time_deriv.momentum[1:-1] = (
        -1/dx * (momentum_flux[1:] - momentum_flux[:-1])
        + accel_grav[1:-1] * fields.height[1:-1]
        - gamma * fields.momentum[1:-1]
        + lax_friedrichs_diff(fields.momentum)
    )
    # fields_time_deriv.momentum[0] = fields_time_deriv.momentum[1]
    # fields_time_deriv.momentum[-1] = fields_time_deriv.momentum[-2]
    
    return fields_time_deriv.pack()


print('Solving PDE... ')
init_fields = Fields.zeros()
slice_ = slice(10 * n // 20, 14 * n // 20)
init_fields.height[slice_] = terrain.max() + 0.3 - terrain[slice_]
sol = scipy.integrate.solve_ivp(
    time_deriv_func,
    (0, t_max),
    init_fields.pack(),
    # method='LSODA',
    # method='BDF',
    # method='RK23',
    # method='DOP853',
)
print(f'Done, num steps: {sol.t.shape[0]}', flush=True)

sol_interp = scipy.interpolate.Akima1DInterpolator(sol.t, sol.y.T)

frame_rate_hz = 20
num_frames = 150
with mpl.autovideo('video_v2.mp4', 3, frame_rate_hz=frame_rate_hz, size_inches=(6, 6), sharex=True) as av:
    for i in progressbar.progressbar(range(num_frames)):
        t = i / num_frames * t_max
        fields = Fields.unpack(sol_interp(t))
        fields_time_deriv = Fields.unpack(time_deriv_func(t, fields.pack()))
        with av.next_frame() as ap:
            # ap.plot(x, fields.height)
            min_terrain = terrain.min()
            _ = ap.ax.fill_between(x, min_terrain, terrain, label='terrain', color='sienna')
            _ = ap.ax.fill_between(x, terrain, terrain + fields.height, label='water', color='dodgerblue')
            # if t < t_rain_stop:
            #     _ = ap.ax.fill_between(x, terrain + fields.height, terrain.max() + 0.1, label='water', color='lightgray')
            ap.set(xlim=[0, 1], ylim=[None, 0.3])

            ap = ap.next()
            ap.plot(x, fields.momentum, label='$p$ (momentum density)')
            # ap.plot(x, fields.momentum / (fields.height + 1e-9), label='v')
            ap.set(ylim=[-.075, .075])
            ap.legend()
            
            ap = ap.next()
            ap.plot(x, fields_time_deriv.momentum, label='$dp/dt$', color=mpl.COLORS[3])
            ap.set(ylim=[-.3, .3], xlabel='$x$')
            ap.legend()

Solving PDE... 
Done, num steps: 573


100% (150 of 150) |######################| Elapsed Time: 0:00:48 Time:  0:00:48
